#K-Nearest Neighbors (KNN) Classification


1. **Setup and Data Loading**

---

*   **Imports**: Libraries like pandas, numpy, and scikit-learn are imported for data handling, preprocessing, and modeling.
*   **Data Loading**: The dataset magic04.data is loaded with columns describing features of particle collisions (e.g., fLength, fWidth) and a binary class (1=Gamma, 0=Hadron). The target is converted to binary integers for classification.







In [ ]:
# Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
# Load data
cols = ["fLength", "fWidth", "fSize", "fConc", "fConcl", "fAsym",
        "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv("magic04.data", names=cols)
df["class"] = (df["class"] == "g").astype(int)  # Binary: 1=Gamma, 0=Hadron


2. **Data Preparation**
---

*  **prepare_data() Function:**

*   - **Standardization**: Features are standardized using StandardScaler() to ensure equal weight in distance calculations.
*   - **Oversampling**: If oversample=True, the minority class is oversampled (using RandomOverSampler) to address class imbalance.

*   **Train-Validation**
- - **Test Split**: Data is shuffled and split into 60% training, 20% validation, and 20% test sets. The training set is oversampled for balance.










In [ ]:
# Data Preparation
def prepare_data(df, oversample=False):
    X = df[cols[:-1]].values
    y = df[cols[-1]].values
    X = StandardScaler().fit_transform(X)  # Standardize features
    if oversample:
        from imblearn.over_sampling import RandomOverSampler
        X, y = RandomOverSampler().fit_resample(X, y)
    return X, y


In [ ]:
# Split data
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
X_train, y_train = prepare_data(train, oversample=True)  # Balance classes
X_test, y_test = prepare_data(test)


3. **KNN Training and Evaluation**

---

- **Model Initialization**: KNeighborsClassifier(n_neighbors=5) creates a KNN model with *k*=5.

- **Training**: knn.fit() stores the standardized training data.

- **Prediction & Evaluation**: Predictions on the test set (X_test) are evaluated using classification_report, which outputs precision, recall, and F1-score.

In [ ]:
# Train and Evaluate KNN
knn = KNeighborsClassifier(n_neighbors=5)  # Try k=1, 3, 5, etc.
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))


4. **Decision Boundary Visualization (2D PCA Projection)**

- **Dimensionality Reduction**: PCA reduces features to 2 principal components for visualization.

- **Meshgrid Creation**: A grid of points is generated to cover the feature space, and KNN predicts class labels for each point.

- **Plotting**:

- - contourf() draws the decision boundary based on grid predictions.

- - scatter() plots the PCA-reduced training points, colored by class.

- - The plot helps visualize how KNN classifies regions based on neighbor votes.

In [ ]:
# Visualize Decision Boundary (2D PCA Projection)
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Reduce features to 2D using PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Step 2: Retrain KNN on 2D data (for visualization)
knn_pca = KNeighborsClassifier(n_neighbors=5)
knn_pca.fit(X_train_pca, y_train)

# Step 3: Create a meshgrid for decision boundary
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = knn_pca.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Step 4: Plot decision boundary and data points
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.Paired)
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train,
            edgecolor='k', s=50, cmap=plt.cm.Paired)
plt.title("KNN Decision Boundary (PCA-Reduced Data)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar(label="Class (0=Hadron, 1=Gamma)")
plt.show()